In [ ]:
# Group 3

In [ ]:
import pandas as pd
import statsmodels as sm
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [ ]:
# load dataset
df = pd.read_csv('/Users/muhammadraza/Documents/GitHub/BIPM/Data Science/data/Dataset_Melbourne.csv', index_col=None)

In [ ]:
df.info()

In [ ]:
# split where price is not na (~0.1)
# replace msising values with median or average and then see the impact on the metrics
# 

In [ ]:
# Some features are missing for a lot of the data.
# Most are floats and some are objects

# Dropping the nulls

df = df.dropna(subset = ['Price'])

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27247 entries, 1 to 34856
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         27247 non-null  object 
 1   Address        27247 non-null  object 
 2   Rooms          27247 non-null  int64  
 3   Type           27247 non-null  object 
 4   Price          27247 non-null  float64
 5   Method         27247 non-null  object 
 6   SellerG        27247 non-null  object 
 7   Date           27247 non-null  object 
 8   Distance       27246 non-null  float64
 9   Postcode       27246 non-null  float64
 10  Bedroom2       20806 non-null  float64
 11  Bathroom       20800 non-null  float64
 12  Car            20423 non-null  float64
 13  Landsize       17982 non-null  float64
 14  BuildingArea   10656 non-null  float64
 15  YearBuilt      12084 non-null  float64
 16  CouncilArea    27244 non-null  object 
 17  Lattitude      20993 non-null  float64
 18  Longtitude 

In [ ]:
X = df[['Rooms', 'Distance', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt']]
y = df['Price']

In [21]:
from sklearn import set_config
set_config(transform_output="pandas")

imputer_median = SimpleImputer(strategy="median", add_indicator=True)

ct = ColumnTransformer(
    [('imputer', imputer_median, ['Price'])],
    remainder='passthrough'
)

transformed_df = ct.fit_transform(df)

In [22]:
transformed_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27247 entries, 1 to 34856
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   imputer__Price            27247 non-null  float64
 1   remainder__Suburb         27247 non-null  object 
 2   remainder__Address        27247 non-null  object 
 3   remainder__Rooms          27247 non-null  int64  
 4   remainder__Type           27247 non-null  object 
 5   remainder__Method         27247 non-null  object 
 6   remainder__SellerG        27247 non-null  object 
 7   remainder__Date           27247 non-null  object 
 8   remainder__Distance       27246 non-null  float64
 9   remainder__Postcode       27246 non-null  float64
 10  remainder__Bedroom2       20806 non-null  float64
 11  remainder__Bathroom       20800 non-null  float64
 12  remainder__Car            20423 non-null  float64
 13  remainder__Landsize       17982 non-null  float64
 14  remainder__

In [23]:
X = transformed_df[['Rooms', 'Distance', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt']]
y = transformed_df['Price']

KeyError: "None of [Index(['Rooms', 'Distance', 'Bedroom2', 'Bathroom', 'Car', 'Landsize',\n       'BuildingArea', 'YearBuilt'],\n      dtype='object')] are in the [columns]"

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1)